In [1]:
signuature = ''

## Daily Aggregrage
- dictionary is done
- aggreage both domain, tag and tag_gp

### 0. Initialization

In [2]:
import cx_Oracle
import pandas as pd
from smtutil import common
from smtutil import hiveutil
from datetime import datetime
import numpy as np
import csv, pickle

Connect to Oracle Successfully
Connect to Oracle Successfully


In [3]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [4]:
sc, spark = hiveutil.get_sc_spark()
hiveutil.register_common_udf(spark)

In [5]:
spark.sql("""
refresh PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM
""")

DataFrame[]

In [6]:
dates_to_loaded = spark.sql("""
with t_loaded as (
    select distinct
        date_id
    from weblog.t_domain_daily
), t_legal as (
    select distinct
        date_id
    from PRD_BIZ_SUMM_VW.VW_IDP_SSL_SED_CALL_SUMM
)
select 
    t_legal.date_id
from t_legal
left join t_loaded
    on t_legal.date_id = t_loaded.date_id
where t_loaded.date_id is null
and t_legal.date_id >= 20200101
order by t_legal.date_id desc
limit 4
""").toPandas().date_id
dates_to_loaded = list(dates_to_loaded)
dates_to_loaded

[20230109, 20220828, 20220827, 20220826]

### 1. Aggregation

In [7]:
spark.sql("refresh weblog.t_domain_daily")
spark.sql("""
with t as (
    select distinct
        date_id
    from weblog.t_domain_daily
)
select
    left(date_id, 6) as month_id,
    count(*) as days
from t
group by month_id
order by month_id
""").toPandas()

month_id  days
0    202001    31
1    202002    29
2    202003    31
3    202004    30
4    202005    31
5    202006    30
6    202007    31
7    202008    31
8    202009    30
9    202010    31
10   202011    30
11   202012    31
12   202101    31
13   202102    28
14   202103    31
15   202104    30
16   202105    31
17   202106    30
18   202107    31
19   202108    31
20   202109    30
21   202110    31
22   202111    30
23   202112    31
24   202201    15
25   202208     3
26   202209    30
27   202210    31
28   202211    30
29   202212    31
30   202301     8

In [8]:
spark.sql("refresh weblog.t_tag_daily")
spark.sql("""
with t as (
    select distinct
        date_id
    from weblog.t_tag_daily
)
select
    left(date_id, 6) as month_id,
    count(*) as days
from t
group by month_id
order by month_id
""").toPandas()

month_id  days
0    202001    31
1    202002    29
2    202003    31
3    202004    30
4    202005    31
5    202006    30
6    202007    31
7    202008    31
8    202009    30
9    202010    31
10   202011    30
11   202012    31
12   202101    31
13   202102    28
14   202103    31
15   202104    30
16   202105    31
17   202106    28
18   202107    31
19   202108    31
20   202109    30
21   202110    31
22   202111    30
23   202112    31
24   202201    15
25   202208     3
26   202209     2
27   202210     3

In [9]:
spark.sql("refresh weblog.t_tag_gp_daily")
spark.sql("""
with t as (
    select distinct
        date_id
    from weblog.t_tag_gp_daily
)
select
    left(date_id, 6) as month_id,
    count(*) as days
from t
group by month_id
order by month_id
""").toPandas()

month_id  days
0    202001    31
1    202002    29
2    202003    31
3    202004    30
4    202005    31
5    202006    30
6    202007    31
7    202008    31
8    202009    30
9    202010    31
10   202011    30
11   202012    31
12   202101    31
13   202102    28
14   202103    31
15   202104    30
16   202105    31
17   202106    28
18   202107    31
19   202108    31
20   202109    30
21   202110    31
22   202111    30
23   202112    31
24   202201    14
25   202208     3
26   202209     2
27   202210     3

#### 1.1. Aggregate Domain

In [10]:
dates_to_loaded

[20230109, 20220828, 20220827, 20220826]

In [11]:
for date_id in dates_to_loaded: # 20 mins
    print(date_id, datetime.now())
    spark.sql(f"alter table weblog.t_domain_daily drop if exists partition (date_id={date_id})")
    spark.sql("""
    select
        subr_num,
        domain_index as agg_value,
        'domain' as agg_level,
        0 as in_byte,
        0 as out_byte,
        byte_count,
        total_hit,
        0 as duration,
        date_id
    from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
    join weblog.t_domain_index
        on vw_idp_ssl_sed_call_summ.ssl_cert_domain = t_domain_index.domain
    where date_id = {date_id}
    """.format(
        date_id=date_id
    ))\
        .coalesce(10)\
        .write.mode('append')\
        .partitionBy('date_id')\
        .saveAsTable('weblog.t_domain_daily')
print(datetime.now())

20230109 2023-01-10 15:41:53.681380
20220828 2023-01-10 15:43:28.645825
20220827 2023-01-10 15:43:34.022874
20220826 2023-01-10 15:43:39.284542
2023-01-10 15:43:44.403591


In [12]:
# %%javascript
# alert('done')

### 1.2. Aggregate Tag

In [13]:
spark.sql("""
select *
from weblog.t_tag_dictionary
limit 2
""").toPandas()

Empty DataFrame
Columns: [tag_gp, tag, domain, agg_level, agg_value, default_aggregation]
Index: []

In [14]:
for date_id in dates_to_loaded: # 20 mins        
    print(date_id, datetime.now()) # 1 min
    spark.sql(f"alter table weblog.t_tag_daily drop if exists partition (date_id={date_id})")
    spark.sql("""
    with t_tag_dictionary as (
        select
            t_tag_dictionary.tag_gp,
            t_tag_dictionary.tag,

            t_tag_dictionary.agg_value,
            t_tag_dictionary.agg_level,

            t_domain_dictionary.agg_value as domain_index
        from weblog.t_tag_dictionary
        join weblog.t_domain_dictionary
            on t_tag_dictionary.tag_gp = t_domain_dictionary.tag_gp
            and t_tag_dictionary.tag = t_domain_dictionary.tag
        where t_tag_dictionary.default_aggregation = 1
    )
    select
        subr_num,
        t_tag_dictionary.agg_value,
        t_tag_dictionary.agg_level,
        sum(in_byte) as in_byte,
        sum(out_byte) as out_byte,
        sum(byte_count) as byte_count,
        sum(total_hit) as total_hit,
        0 as duration,
        date_id
    from weblog.t_domain_daily
    join t_tag_dictionary
        on t_tag_dictionary.domain_index = t_domain_daily.agg_value
    where date_id = '{date_id}'
    group by subr_num, date_id, t_tag_dictionary.agg_value, t_tag_dictionary.agg_level
    """.format(
        date_id=date_id
    ))\
        .coalesce(10)\
        .write.mode('append')\
        .partitionBy('date_id')\
        .saveAsTable('weblog.t_tag_daily')
print(datetime.now())

20230109 2023-01-10 15:43:44.546536
20220828 2023-01-10 15:44:08.253674
20220827 2023-01-10 15:44:13.213706
20220826 2023-01-10 15:44:18.076015
2023-01-10 15:44:22.984748


In [15]:
# %%javascript
# alert('done')

### 1.3. Aggregate Tag Group

In [16]:
for date_id in dates_to_loaded: # 15 mins    
    print(date_id, datetime.now())
    
    spark.sql(f"alter table weblog.t_tag_gp_daily drop if exists partition (date_id={date_id})")
    spark.sql("""
    with t_tag_dictionary as (
        select
            t_tag_gp_dictionary.tag_gp,
            t_tag_gp_dictionary.tag,

            t_tag_gp_dictionary.agg_value,
            t_tag_gp_dictionary.agg_level,

            t_domain_dictionary.agg_value as domain_index
        from weblog.t_tag_gp_dictionary
        join weblog.t_domain_dictionary
            on t_tag_gp_dictionary.tag_gp = t_domain_dictionary.tag_gp
        where t_tag_gp_dictionary.default_aggregation = 1
    )
    select
        subr_num,
        t_tag_dictionary.agg_value,
        t_tag_dictionary.agg_level,
        sum(in_byte) as in_byte,
        sum(out_byte) as out_byte,
        sum(byte_count) as byte_count,
        sum(total_hit) as total_hit,
        0 as duration,
        date_id
    from weblog.t_domain_daily
    join t_tag_dictionary
        on t_tag_dictionary.domain_index = t_domain_daily.agg_value
    where date_id = '{date_id}'
    group by subr_num, date_id, t_tag_dictionary.agg_value, t_tag_dictionary.agg_level
    """.format(
        date_id=date_id
    ))\
        .coalesce(10)\
        .write.mode('append')\
        .partitionBy('date_id')\
        .saveAsTable('weblog.t_tag_gp_daily')
print(datetime.now())

20230109 2023-01-10 15:44:23.031064
20220828 2023-01-10 15:44:48.050450
20220827 2023-01-10 15:44:52.680804
20220826 2023-01-10 15:44:57.116321
2023-01-10 15:45:01.520620


In [17]:
# %%javascript
# alert('done')

In [18]:
spark.stop()

In [19]:
sc.stop()